In [1]:
cd drive/MyDrive/kobert-wellness-chatbot/

/content/drive/MyDrive/kobert-wellness-chatbot


In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661750 sha256=f9b5088d0733148e676386a50f2e7472ddb1b987db4944132198044d60ab64eb
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86

In [4]:
import os
import torch
from transformers import AdamW, BertModel
from model.classifier import BERTClassifier, kobert_input
from model.dataloader import WellnessTextClassificationDataset
from kobert_tokenizer import KoBERTTokenizer
import torch.nn as nn
import random

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [5]:
import openpyxl
from openpyxl import Workbook, load_workbook

In [6]:
# 감정 카테고리 로드
def load_emotion_category():
    # 카테고리 파일 경로 설정
    category_path = './data/wellness_dialog_category_감정.txt'

    # 카테고리 파일 열기
    c_f = open(category_path, 'r')
    # 파일의 모든 라인 읽기
    category_lines = c_f.readlines()

    # 각 라인을 처리하여 딕셔너리에 저장
    category = {}
    for line_num, line_data in enumerate(category_lines):
        data = line_data.split('    ')
        category[data[1][:-1]] = data[0]
    return category

In [7]:
if __name__ == "__main__":
    # 디바이스 설정
    ctx = "cuda" if torch.cuda.is_available() else "cpu"
    device = torch.device(ctx)
    # 체크포인트 로드
    checkpoint_path = "./checkpoint"
    save_ckpt_path = checkpoint_path + "/kobert-wellness-text-classification_감정.pth"
    checkpoint = torch.load(save_ckpt_path, map_location=device)
    # KoBERT 모델 로드
    bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
    # 모델 초기화
    model = BERTClassifier(bertmodel)
    # 모델 상태 딕셔너리 로드
    model.load_state_dict(checkpoint['model_state_dict'])
    # 모델을 디바이스에 로드
    model.to(ctx)
    model.eval()

    # 토크나이저 로드
    tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

    # 감정 카테고리 로드
    category = load_emotion_category()

    # 감성 대화 말뭉치 로드
    corpus_file = './data/감성대화말뭉치_감정분류.xlsx'
    wb = load_workbook(filename=corpus_file)
    ws = wb[wb.sheetnames[0]]
    first_line = 0
    # 각 행을 순회
    for row in ws.iter_rows():
        if first_line == 0:
            first_line = 1
            continue
        # 문장 데이터 가져오기
        sent = row[2].value
        # 입력 데이터 생성
        data = kobert_input(tokenizer, sent, device, 512)
        # 모델에 입력하여 출력값 얻기
        output = model(**data)

        # 로짓 값 추출
        logit = output[0]
        # 소프트맥스 적용
        softmax_logit = torch.softmax(logit, dim=-1)
        softmax_logit = softmax_logit.squeeze()

        # 최댓값 인덱스 및 값 추출
        max_index = torch.argmax(softmax_logit).item()
        max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

        # 예측된 카테고리 추출
        cate_pred = category[str(max_index)]

        # 예측된 카테고리를 4번째 열에 저장
        row[3].value = cate_pred

    # 변경된 내용을 파일에 저장
    wb.save(corpus_file)

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
print(category)

{'0': '무난함', '1': '걱정', '2': '불안', '3': '두려움', '4': '고립감', '5': '외로움', '6': '속상함', '7': '예민함', '8': '당황', '9': '부끄러움', '10': '놀람', '11': '혼란스러움', '12': '답답함', '13': '괴로움', '14': '지침', '15': '무기력', '16': '허무함', '17': '좌절감', '18': '자괴감', '19': '후회', '20': '죄책감', '21': '억울함', '22': '분노', '23': '혐오', '24': '불만', '25': '짜증', '26': '불쾌함', '27': '불편함', '28': '슬픔', '29': '우울함', '30': '기쁨', '31': '뿌듯함', '32': '설렘', '33': '안도', '34': '흥분', '35': '감사함', '36': '여유로움', '37': '만족스러움', '38': '자신하는', '39': '편안함'}
